In [35]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import pickle
from selenium import webdriver
from lxml import etree
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.chrome.options import Options
from pyvirtualdisplay import Display


In [13]:
def crawl_region_pyql(region_list,url='http://search.qyer.com/all/271.html', pages=5):
    opts = Options()
    # opts.add_argument("--incognito")  # 使用無痕模式

    ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    opts.add_argument("user-agent={}".format(ua))         # 使用偽造的 user-agent

    prefs = {'download.default_directory': './crawl_pyql_data'}
    opts.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome('/shared_home/willie/chromedriver',options=opts)
    browser.get(url)
    browser.implicitly_wait(10)
    current_window = browser.current_window_handle
    
    attraction_regions = {}
    
    for region in region_list:
        browser.find_element_by_xpath("//div[@class='q-header-search']//input[@name='keyword']").send_keys(region)
        browser.find_element_by_xpath("//button[@class='btn-search']").click()
        browser.find_element_by_xpath("//a[contains(text(),'景点美食')]").click()

        attraction_list = []
        all_windows = browser.window_handles

        # 切換 window
        for window in all_windows:
            if window != current_window:
                browser.switch_to.window(window)
        
        doc = pq(browser.current_url,encoding="utf-8")
        all_pages = int(doc('.ui_page').text().split(' ')[-2].replace('.',''))
        crawl_pages = min(all_pages,pages)
        
        # 拿attractions for the first page
        attractions = doc('.plcPoilistModules .title.fontYaHei').text().replace('\xa0','')
        attraction_list.append(attractions)
        
        for p in range(crawl_pages-1):
            time.sleep(5)
            browser.find_element_by_css_selector('.ui_page_item.ui_page_next').click()
            
            # 拿attractions
            element = browser.find_element_by_css_selector('.plcPoilistModules')
            doc = pq(element.get_attribute('innerHTML'))
            attractions = doc('.title.fontYaHei').text()#.replace('\xa0','')
            
            attraction_list.append(attractions)
            
        attraction_regions[region] = attraction_list

        # 關窗口與切換窗口
        browser.close()    
        browser.switch_to.window(current_window)
    browser.quit()
    return attraction_regions

In [9]:
def crawl_region_trip(region_list,url_temp, pages=5):
    display = Display(visible=0, size=(800, 800))  
    display.start()
    opts = Options()
    opts.add_argument("--incognito")  # 使用無痕模式

    ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    opts.add_argument("user-agent={}".format(ua))         # 使用偽造的 user-agent

    prefs = {'download.default_directory': './crawl_tripAdvisior_data'}
    opts.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome('/shared_home/willie/chromedriver',options=opts)
    
    attraction_regions = {}
    headers = {'x-requested-with': 'XMLHttpRequest'}
    
    # check current progress
    for file in os.listdir('./crawl_tripAdvisior_data'):
        current_file = 0
        if('attraction_regions_' in file and 'all' not in file):
            num = file.replace('attraction_regions_','').replace('.pkl','')
            current_file = max(current_file,int(num))
        
    for i,region in enumerate(region_list):
        if(i <= current_file):
            print(region,'is crawled')
            continue
            
        url = url_temp.replace('location',region)
        
        attraction_list = []
        
        browser.get(url)
        time.sleep(3)
        try:
            browser.find_element_by_css_selector(".ui_link.show-text").click() # 更多
        except:
            print('No more!!')
        time.sleep(3)
        
        # Get attractions
        doc = pq(browser.page_source)
        tags = doc('.thumbnail-overlay-tag')
        title = doc('.result-title')

        for j,d in enumerate(title.items()):
            try:
                attraction_list.append((d.text(),tags[j].text))
            except:
                attraction_list.append((d.text(),''))

        all_pages = int(doc('.pageNum ').text().split(' ')[-1])
        crawl_pages = min(all_pages,pages)
        
        for p in range(crawl_pages-1):
            
            if(p%5 == 0):
                print(region,p)
                
            browser.find_element_by_xpath("//a[contains(@class,'ui_button nav next primary')]").click()
            time.sleep(5)
            
            # 拿attractions
            doc = pq(browser.page_source)

            tags = doc('.thumbnail-overlay-tag')
            title = doc('.result-title')
            for j,d in enumerate(title.items()):
                try:
                    attraction_list.append((d.text(),tags[j].text))
                except:
                    attraction_list.append((d.text(),''))

        attraction_regions[region] = attraction_list

        # save data
        with open(f'./crawl_tripAdvisior_data/attraction_regions_{i}.pkl','wb') as f:
            pickle.dump(attraction_regions,f)
            
    # save all data
    with open(f'./crawl_tripAdvisior_data/attraction_regions_all.pkl','wb') as f:
        pickle.dump(attraction_regions,f)        
    browser.quit()
    print("Don't look at me!")
    time.sleep(5)
    print("Yes, jusy you. Fuck OFF!!!")
    
    return attraction_regions


In [15]:
with open('regions.pkl','rb') as f:
    regions = pickle.load(f)

In [54]:
with open('./crawl_tripAdvisior_data/attraction_regions_all.pkl','rb') as f:
    attraction_regions = pickle.load(f)

In [55]:
regions

array(['台北', '台北', '台北', ..., '澎湖', '澎湖', '澎湖'], dtype='<U2')

In [56]:
attraction_regions

{'台北': [('臺北捷運', '公共交通系統'),
  ('台北101', '建築物'),
  ('桃園機場捷運線', '公共交通系統'),
  ('台北101觀景台', '建築物'),
  ('象山自然步道', '景點和地標'),
  ('台北地下街', '購物中心'),
  ('微風台北車站', '特色和禮品商店'),
  ('臺北101跨年煙火', '景點'),
  ('台北車站', '公共交通系統'),
  ('西門町', '地區'),
  ('貓空纜車', '電車'),
  ('台北小巨蛋', '競技場和體育場館'),
  ('台北賓館', '建築物'),
  ('陽明山國家公園', '史跡徒步區'),
  ('MyTaiwanTour Travel Services', '潛水和浮潛'),
  ('京站時尚廣場', '購物中心'),
  ('TourMeAway - Free Walking Tour In Taipei', '觀光遊覽'),
  ('台北國際書展', '景點'),
  ('龍山寺', '宗教景點'),
  ('台北車站 (桃園捷運)', '公共交通系統'),
  ('台北植物園', '花園'),
  ('大安森林公園', '公園'),
  ('新光三越 台北站前店', '特色和禮品商店'),
  ('台北孔廟', '古蹟'),
  ('貓空山', '電車'),
  ('迪化街', '特色和禮品商店'),
  ('北投溫泉', '溫泉和間歇泉'),
  ('台北國際會議中心', '會議中心'),
  ('台北當代藝術館', '藝術博物館'),
  ('新光三越 台北信義新天地', '特色和禮品商店'),
  ('台灣高鐵', '公共交通系統'),
  ('桃園捷運', '公共交通系統'),
  ('國立故宮博物院', '歷史博物館'),
  ('國光客運', '公共交通系統'),
  ('士林觀光夜市', '跳蚤市場和街頭市集'),
  ('二二八和平紀念公園', '古蹟'),
  ('饒河街觀光夜市', '跳蚤市場和街頭市集'),
  ('中正紀念堂', '紀念碑和雕像'),
  ('來去福爾摩沙', '觀光遊覽'),
  ('台北探索館', '特色博物館'),
  ('Taipei Eats', '步行遊覽'),
  ('國父紀念

In [61]:
types = []
attractions = []
regions = []
attr_type =[]
for k,v in attraction_regions.items():
    for l in v:
        types.append(l[1])
        attractions.append(l[0])
        regions.append(k)
        attr_type.append(l)

types = np.array(types)
types_unique = np.unique(types)
attractions = np.array(attractions)
attractions_unique = np.unique(attractions)
regions = np.array(regions)
regions_unique = np.unique(regions)
attr_type = np.array(attr_type)
attr_type_unique = np.unique(attr_type,axis=1)

In [65]:
print(types.shape)
print(types_unique.shape)
print(attractions.shape)
print(attractions_unique.shape)
print(regions.shape)
print(regions_unique.shape)
print(attr_type.shape)
print(attr_type_unique.shape)

(7521,)
(155,)
(7521,)
(6242,)
(7521,)
(18,)
(7521, 2)
(7521, 2)


In [60]:
types_unique 

array(['', 'Spa', '交響樂', '健行和露營遊覽', '健身中心', '兒童博物館', '公共交通系統', '公園',
       '公車服務', '其他', '其他戶外活動', '其他餐飲', '刺激極限遊覽', '劇場', '劇場和表演', '動物園',
       '包船釣魚和遊覽', '卡拉 OK 酒吧', '參觀工廠', '叢林飛行和空中冒險公園', '古代遺址', '古董店', '古蹟',
       '史跡徒步區', '咖啡和茶遊覽', '咖啡館', '品嚐啤酒和遊覽', '品酒之旅 ', '品酒酒吧', '商店', '啤酒廠',
       '單車專用道', '單車遊覽', '噴泉', '國家公園', '圖書館', '地區', '地質構造', '城堡', '城市遊覽',
       '墓園', '大洞穴和洞窟', '宗教景點', '山脈', '山谷', '島嶼', '峽谷', '工廠暢貨中心', '市政中心',
       '建築物', '慢跑小徑和步道', '戰場', '政府建築', '教堂和大教堂', '教育景點', '文化遊覽',
       '文學、藝術和音樂遊覽', '晚宴劇場', '景觀徒步區', '景點', '景點和地標', '會議中心', '森林', '橋樑',
       '步行遊覽', '歷史博物館', '歷史古蹟遊覽', '水上樂園', '水上運動', '水域', '水壩', '水族館', '沙漠',
       '海灘', '海灘和游泳池俱樂部', '渡輪', '溫泉 Spa', '溫泉和間歇泉', '溫泉度假村', '滑翔傘和滑翔翼',
       '滑雪和滑雪板區', '潛水和浮潛', '瀑布', '火山', '烹飪課程', '熱氣球之旅', '燈塔', '特色博物館',
       '特色和禮品商店', '獨木舟', '珊瑚礁', '瑜伽與皮拉提斯', '生態遊覽', '登山步道', '登山遊覽', '百貨公司',
       '瞭望台', '碼頭和木板人行道', '礦區', '神秘景點', '私人遊覽', '科學博物館', '租船', '空中遊覽',
       '競技場和體育場館', '紀念碑和雕像', '美術館', '美食遊覽', '自助式導覽遊覽和租賃', '自然和野生

In [49]:
types

array(['公共交通系統', '建築物', '公共交通系統', ..., '', '', ''], dtype='<U12')